<a href="https://colab.research.google.com/github/datvodinh10/recurrent-ppo/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!lscpu | grep 'Model name'

In [ ]:
!git clone https://github.com/datvodinh10/recurrent-ppo.git
%cd recurrent-ppo

In [ ]:
from model.trainer import *
%cd ENV
import env

In [ ]:
game_name = "Splendor_v3"
env.make(game_name)
writer_path = f"/content/runs/{game_name}/"

In [ ]:
import numpy as np
from numba import njit

def bot_max_eps_length(state, perData):
    validActions = env.getValidActions(state)
    arr_action = np.where(validActions == 1)[0]
    idx = np.random.randint(0, arr_action.shape[0])
    perData[0]+=1
    if env.getReward(state)!=-1:
        if perData[0] > perData[1]:
            perData[1] = perData[0]
        perData[0] = 0
    return arr_action[idx], perData
per = np.array([0,0])
env.run(bot_max_eps_length,100,per,1)

max_eps_length = per[1] + 50
max_eps_length

In [ ]:
config = {
    "critic_coef": 1,
    "entropy_coef": 0.001,
    "num_epochs": 2,
    "max_eps_length":max_eps_length,
    "seq_length":16,
    "hidden_size":256,
    "embed_size": 256,
    "lr": 2.5e-4,
    "num_game_per_batch":128,
    "gamma":0.99,
    "gae_lambda":0.95,
    "policy_kl_range":0.0008,
    "policy_params": 20,
    "value_clip": 1.0,
    "max_grad_norm": 0.5,
    "n_mini_batch": 4,
    "rewards": [-1,1,0], #[lose,win,not_done]
    "set_detect_anomaly": True,
    "normalize_advantage": False,
    "loss_type":"TrulyPPO"

}


In [ ]:
trainer = Trainer(config=config,env=env,writer_path = writer_path)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/runs/

In [ ]:
trainer.train()